# Introduzione

Tutti i giorni ci confrontiamo con delle situazioni i cui risultati sono incerti ad esempio questo farmaco può aiutare a risolvere un problema, questa è la moglie/marito giusta/o per me?

Sulla seconda domanda siamo di fronte ad un problema in cui le variabili in gioco sono molte, ma ci sono alcuni casi in cui possiamo estrarre il massimo della conoscenda dalle informazioni in nostro possesso.

Ad esempio possiamo rispondere alla domanda "Questo farmaco può causare nausea ?" rispondendo in questo modo "Tre pazienti su mille hanno avuto problemi di nausea subito dopo aver preso quel farmaco". 

La statistica generalmente aiuta a impostare le seguenti problematiche:

+ Chiarire la domanda
+ Identificare la variabile da misurare
+ Determinare la dimensione del campione
+ Descrivere variazioni
+ Dare risposte quantitative sui parametri stimati
+ Fare delle previsioni utilizzando i dati in nostro possesso



In [1]:
import re
import pandas as pd
import numpy as np

def readStataDct(dct_file, **options):
    type_map = dict(byte=int, int=int, long=int, float=float, 
                    double=float, numeric=float)

    var_info = []
    with open(dct_file, **options) as f:
        for line in f:
            match = re.search( r'_column\(([^)]*)\)', line)
            if not match:
                continue
            start = int(match.group(1)) - 1
            t = line.split()
            vtype, name, fstring = t[1:4]
            name = name.lower()
            if vtype.startswith('str'):
                vtype = str
            else:
                vtype = type_map[vtype]
            long_desc = ' '.join(t[4:]).strip('"')
            length = int(re.findall("\d+", fstring)[0])
            var_info.append((start,length, vtype, name, fstring, long_desc))
        
        columns = ['start','length', 'type', 'name', 'fstring', 'desc']
        variables = pd.DataFrame(var_info, columns=columns)
        variables['end'] = variables['start'] + variables['length']
    return variables

cols = readStataDct('./data/2006_2010_FemPregSetup.dct',encoding='iso-8859-1')

In [2]:
def readStataDataset(filename,variables, **options):
    colspecs = variables[['start', 'end']]
    colspecs = colspecs.astype(np.int32).values.tolist()
    names = variables['name']
    
    df = pd.read_fwf(filename,colspecs=colspecs, names=names,**options)
    
    return df
    

In [16]:
df = readStataDataset('./data/2006_2010_FemPreg.dat.gz',cols,compression='gzip')

for col in df.columns:
    print(col)

caseid
pregordr
howpreg_n
howpreg_p
moscurrp
nowprgdk
pregend1
pregend2
pregend3
howenddk
nbrnaliv
multbrth
bornaliv
datprgen_m
datprgen_y
cmotpreg
ageatend
gestasun_m
gestasun_w
wksgest
mosgest
dk1gest
dk2gest
dk3gest
bpa_bdscheck1
bpa_bdscheck2
bpa_bdscheck3
babysex1
birthwgt_lb1
birthwgt_oz1
lobthwgt1
babysex2
birthwgt_lb2
birthwgt_oz2
lobthwgt2
babysex3
birthwgt_lb3
birthwgt_oz3
lobthwgt3
babydob_m
babydob_y
cmbabdob
kidage
hpagelb
birthplc
paybirth1
paybirth2
paybirth3
paybirth4
csecprim
csecmed1
csecmed2
csecmed3
csecmed4
csecmed5
csecplan
knewpreg
trimestr
ltrimest
priorsmk
postsmks
npostsmk
getprena
bgnprena
pnctrim
lpnctri
workpreg
workborn
didwork
matweeks
weeksdk
matleave
livehere1
alivenow1
whendied_m1
whendied_y1
cmkidied1
whenleft_m1
whenleft_y1
cmkidlft1
lastage1
wherenow1
legagree1
parenend1
anynurse1
fedsolid1
frsteatd_n1
frsteatd_p1
frsteatd1
quitnurs1
ageqtnur_n1
ageqtnur_p1
ageqtnur1
livehere2
alivenow2
whendied_m2
whendied_y2
cmkidied2
whenleft_m2
whenleft_y2
cmkid

In [21]:
def cleanFemPreg(df):
    df.agepreg /= 100
    na_vals = [97, 98, 99]
    df.birthwgt_lb1.replace(na_vals,np.nan,inplace=True)
    df.birthwgt_oz1.replace(na_vals,np.nan,inplace=True)
        
    df['totalwgt_lb'] = df.birthwgt_lb1 + df.birthwgt_oz1 / 16.0
    
cleanFemPreg(df)

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,pregend3,howenddk,...,secu,sest,cmintvw,cmlstyr,cmjan3yr,cmjan5yr,questyear,quarter,phase,totalwgt_lb
0,39968,1,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,2,105,1323,1311,1285,1261,4,15,2,NaN
1,39968,2,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,2,105,1323,1311,1285,1261,4,15,2,NaN
2,41474,1,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,2,105,1322,1310,1285,1261,4,15,1,NaN
3,33303,1,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,2,105,1289,1277,1249,1225,1,4,1,NaN
4,33303,2,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,...,2,105,1289,1277,1249,1225,1,4,1,8.5


## Analisi delle variabili

Il dataset contiene 287 variabili, per le nostre analisi useremo le seguenti variabili:

* **caseid** è un intero che rappresenta la persona che ha compilato il modulo
* **prglenght** rappresenta il numero di settimane di gestazione
* **outcome** è un intero che rappresenta se il figlio è nato vivo o meno
* **pregorder** è un progressivo che identifica il numero del figlio nato: primo, secondo, terzo
* **birthord** un progressivo per enumerare i bambini nati vivi
* **birthwgt_lb** e **birthwgt_oz** peso in libbre e once del bambino
* **agepreg** età della mamma al parto


In [23]:
df.birthwgt_lb1.value_counts(sort = False)

0.0       13
4.0      359
8.0     2861
7.0     4955
2.0       91
3.0      148
6.0     3385
5.0     1095
9.0      889
11.0      36
10.0     196
1.0       82
12.0      12
13.0       6
16.0       1
Name: birthwgt_lb1, dtype: int64